# LEXICOGRAPHIC-DECOMPOSITION for PolSAR 

Copyright 2025 European Space Agency (ESA)

Licensed under  ESA Software Community License Permissive (Type 3) – v2.4

## Setup 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

### Prerequisities

In [ ]:
import os
import pathlib

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import bioqlk

In [ ]:
import inspect
import IPython.display

### Read image
Import image and read channels.

In [ ]:
basepath = pathlib.Path(
    "/Users/antonio.valentino/data/biomass/first-image/SLCs/MED/BIO_S1_SCS__1S_20250523T102350_20250523T102542_C_G___M___C___T____F001_01_D9DWE9"
)

### Image processing parameters

In [ ]:
winsz = [20, 4]  # Window size for smoothing 
gamma = 1 # Power scaling factor: Default (no scaling is for gamma = 1) If gamma > 1, brighter values are enhanced, if gamma < 1, darker values are enhanced 

threshold = 0.03
# threshold = 0

# Phase correction of 90 degrees as standard, can adjusted per channel: 
# pc_hh = np.deg2rad(-23.75)
# pc_hv = np.deg2rad(113.46)
# pc_vh = np.deg2rad(-89.7)

# pc_hh = np.deg2rad(+23.75)
# pc_hv = np.deg2rad(-113.46)
# pc_vh = np.deg2rad(+89.7)

pc_hh = np.deg2rad(+90)  # np.pi/2
pc_hv = np.deg2rad(+90)  # np.pi/2
pc_vh = np.deg2rad(-90)  # - np.pi/2

# pc_hh = 0
# pc_hv = 0
# pc_vh = 0

## Step 1: Compute Complex Signal from Amplitude and Phase with polar coordinates (a, phi)

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.load_data), language="python")

In [ ]:
hh, hv, vh, vv, meta, gcps = bioqlk.load_data(basepath, return_metadata=True)

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.fix_ical_phase), language="python")

In [ ]:
# Apply the phase correction
hhc, hvc, vhc, vvc = bioqlk.fix_ical_phase(hh, hv, vh, vv, pc_hh, pc_hv, pc_vh, 0.)

In [ ]:
hh.mean(), vv.mean(), hv.mean(), vh.mean()

In [ ]:
hhc.mean(), vvc.mean(), hvc.mean(), vhc.mean()

## Step 2: Lexicographic decomposition: 

* Red channel: |hh| ** 2
* Green channel: 0.5 * (|hv| ** 2 + |vh| ** 2)
* Blue channel: |vv| ** 2

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.lexicographic), language="python")

In [ ]:
k_1, k_2, k_3 = bioqlk.lexicographic(hh, hv, vh, vv, scale=bioqlk.EScale.DB)

In [ ]:
k_1c, k_2c, k_3c = bioqlk.lexicographic(hhc, hvc, vhc, vvc, scale=bioqlk.EScale.DB)

In [ ]:
k_1.mean(), k_2.mean(), k_3.mean()

In [ ]:
k_1c.mean(), k_2c.mean(), k_3c.mean()

In [ ]:
fig, ax = plt.subplots(3, sharex=True)

ax[0].hist(k_1.ravel(), 500, label="k_1")
# ax[0].set_title("k_1")
ax[0].grid()
ax[0].legend()
ax[0].set_title("Lexicographic")

ax[1].hist(k_2.ravel(), 500, label="k_2")
# ax[1].set_title("k_2")
ax[1].grid()
ax[1].legend()

ax[2].hist(k_3.ravel(), 500, label="k_3")
# ax[2].set_title("k_3")
ax[2].grid()
ax[2].legend()
ax[2].set_xlabel("dB")

In [ ]:
fig, ax = plt.subplots(3, sharex=True)

ax[0].hist(k_1c.ravel(), 500, label="k_1")
# ax[0].set_title("k_1")
ax[0].grid()
ax[0].legend()
ax[0].set_title("Lexicographic with polarimetric channels correction")

ax[1].hist(k_2c.ravel(), 500, label="k_2")
# ax[1].set_title("k_2")
ax[1].grid()
ax[1].legend()

ax[2].hist(k_3c.ravel(), 500, label="k_3")
# ax[2].set_title("k_3")
ax[2].grid()
ax[2].legend()
ax[2].set_xlabel("dB")

## Step 3: Image Processing

### 3.1 Smooth

Scipy.ndimage's uniform filter: 
* has no NaNn handling ->  np.nan_to_num 

* mode parameter: `nearest`: (a a a a | a b c d | d d d d) -> The input is extended by replicating the last pixel.

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.smooth_nan), language="python")

In [ ]:
# Smooth 
k_1_smoothed = bioqlk.smooth_nan(k_1, winsz)
k_2_smoothed = bioqlk.smooth_nan(k_2, winsz)
k_3_smoothed = bioqlk.smooth_nan(k_3, winsz)

In [ ]:
# Smooth 
k_1c_smoothed = bioqlk.smooth_nan(k_1c, winsz)
k_2c_smoothed = bioqlk.smooth_nan(k_2c, winsz)
k_3c_smoothed = bioqlk.smooth_nan(k_3c, winsz)

### 3.2 Scaling
* Rescale values into range 0 - 255: x_norm = 255* (x-xmin) / (xmax - xmin) 
* Maybe we should do a percentile normalization to deal with outliers 

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.scale_to_8bits), language="python")

In [ ]:
vmin, vmax = bioqlk.quantile_scaling(k_1_smoothed, threshold=threshold)
k_1_scaled = bioqlk.scale_to_8bits(k_1_smoothed, vmin=vmin, vmax=vmax)
vmin, vmax = bioqlk.quantile_scaling(k_2_smoothed, threshold=threshold)
k_2_scaled = bioqlk.scale_to_8bits(k_2_smoothed, vmin=vmin, vmax=vmax)
vmin, vmax = bioqlk.quantile_scaling(k_3_smoothed, threshold=threshold)
k_3_scaled = bioqlk.scale_to_8bits(k_3_smoothed, vmin=vmin, vmax=vmax)

In [ ]:
vmin, vmax = bioqlk.quantile_scaling(k_1c_smoothed, threshold=threshold)
k_1c_scaled = bioqlk.scale_to_8bits(k_1c_smoothed, vmin=vmin, vmax=vmax)
vmin, vmax = bioqlk.quantile_scaling(k_2c_smoothed, threshold=threshold)
k_2c_scaled = bioqlk.scale_to_8bits(k_2c_smoothed, vmin=vmin, vmax=vmax)
vmin, vmax = bioqlk.quantile_scaling(k_3c_smoothed, threshold=threshold)
k_3c_scaled = bioqlk.scale_to_8bits(k_3c_smoothed, vmin=vmin, vmax=vmax)

### 3.3 Other things we can do 



## Step 4: Write and plot data

### 4.1 Compose RGB image

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.to_rgb_array), language="python")

In [ ]:
rgb = bioqlk.to_rgb_array(k_1_scaled, k_2_scaled, k_3_scaled)

In [ ]:
rgbc = bioqlk.to_rgb_array(k_1c_scaled, k_2c_scaled, k_3c_scaled)

### 4.2 Plot image

In [ ]:
#Rotate image for better visulization in notebook
# rgb_rot = np.rot90(rgb, k=1, axes=(0, 1))
# rgbc_rot = np.rot90(rgbc, k=1, axes=(0, 1))

fig, ax = plt.subplots(1, 2, figsize=(10, 12))
ax[0].imshow(rgb, aspect="auto")  #, origin="upper")  # , interpolation="nearest"
ax[0].axis('off')
ax[1].imshow(rgbc, aspect="auto")  #, origin="upper")  # , interpolation="nearest"
ax[1].axis('off')


### 4.3  Write image to geotiff

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.save_rgb), language="python")

In [ ]:
# out_filename = pathlib.Path(basepath.name + "_pauli_rgb.tiff")
out_filename = basepath.parent / (basepath.name + "_pauli_rgb.tiff")

bioqlk.save_rgb(rgb, out_filename, metadata=meta, gcps=gcps)

### 4.4 Generate KMZ using gdalwarp CLI utility

```
$ gdalwarp -t_srs wgs84 -of KMLSUPEROVERLAY -co format=png ../data_old/BIO_S1_SCS__1S_20250522T095714_20250522T095744_C_G___M___C___T____F001_01_D91R78_pauli_rgb.tiff BIO_S1_SCS__1S_20250522T095714_20250522T095744_C_G___M___C___T____F001_01_D91R78_pauli_rgb.kmz
```

@TODO: do it via rasterio API

In [ ]:
IPython.display.Code(inspect.getsource(bioqlk.save_kmz), language="python")

In [ ]:
if False:
    out_filename_kmz = out_filename.with_suffix(".kmz")
    if out_filename_kmz.exists():
        os.unlink(out_filename_kmz)

    bioqlk.save_kmz(out_filename, out_filename_kmz)

### 4.4: We can interactively view this image on the map with folium
TODO